In [ ]:
!mkdir -p ~/.kaggle

# Move kaggle.json to the correct folder
!mv kaggle.json ~/.kaggle/
!kaggle competitions download -c q-2-superai-ss-5-cp-axtra
!mkdir -p /content/kaggle_competition

In [ ]:
!unzip -q q-2-superai-ss-5-cp-axtra.zip -d /content/kaggle_competition

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#his_sale1 = pd.read_csv('/content/kaggle_competition/historical_sale_2023_1.csv')
#his_sale2 = pd.read_csv('/content/kaggle_competition/historical_sale_2023_2.csv')
#his_sale3 = pd.read_csv('/content/kaggle_competition/historical_sale_2024_1.csv')
#his_sale4 = pd.read_csv('/content/kaggle_competition/historical_sale_2024_2.csv')
his_sale5 = pd.read_csv('/content/kaggle_competition/historical_sale_2025_1.csv')
calendar = pd.read_csv('/content/kaggle_competition/calendar_date.csv')
item = pd.read_csv('/content/kaggle_competition/item_hierarchy.csv')
store = pd.read_csv('/content/kaggle_competition/store_data.csv')
submis = pd.read_csv('/content/kaggle_competition/Q2_sample_submission.csv')

In [ ]:
print(his_sale5.head(15))
print(his_sale5.shape)
print(his_sale5.nunique())
print('\n')
print(calendar.head(5))
print(calendar.shape)
print(calendar.nunique())
print('\n')
print(item.head(5))
print(item.shape)
print(item.nunique())
print('\n')
print(store.head(5))
print(store.shape)
print('\n')
print(submis.head(5))
print(submis.shape)

In [ ]:
print(submis.nunique())

##เตรียม Full data สำหรับการ Rule-Based

---




In [ ]:
def compare_item_ids(df1, df2):
    """
    Compares the 'item_id' column of two pandas DataFrames and identifies items that are not present in both.

    Args:
        df1 (pd.DataFrame): The first DataFrame.
        df2 (pd.DataFrame): The second DataFrame.

    Returns:
        tuple: A tuple containing two pandas Series:
            - items_not_in_df2: Items present in df1 but not in df2.
            - items_not_in_df1: Items present in df2 but not in df1.
    """
    items_df1 = set(df1['item_id'])
    items_df2 = set(df2['item_id'])

    not_in_df2 = pd.Series(list(items_df1 - items_df2), name='item_id')
    not_in_df1 = pd.Series(list(items_df2 - items_df1), name='item_id')
    in_both = list(items_df1.intersection(items_df2))
    print(len(in_both))
    print(not_in_df2.count())
    print(not_in_df1.count())

    return not_in_df2, in_both

not_in_df2, in_both = compare_item_ids(submis, his_sale5)


In [ ]:
# Get the set of stores
stores = set(submis['store_id'])

# Create DataFrames from not_in_df2 and stores
df_items = pd.DataFrame({'item_id': not_in_df2})
df_stores = pd.DataFrame({'store_id': list(stores)})

# Cartesian product using merge with 'cross' join (requires pandas 1.2+)
mapped_df = df_items.merge(df_stores, how='cross')

df_items = pd.DataFrame({'item_id': in_both})

# Cartesian product using merge with 'cross' join (requires pandas 1.2+)
mapped_intersect_df = df_items.merge(df_stores, how='cross')

print(mapped_df.head())
print(mapped_df.shape)  # should be (176640, 2)
print(mapped_intersect_df.head())
print(mapped_intersect_df.shape)  # should be (176640, 2)

mapped_df = mapped_df.merge(
    submis[['item_id', 'store_id']],
    on=['item_id', 'store_id'],
    how='inner'
)

mapped_intersect_df = mapped_intersect_df.merge(
    submis[['item_id', 'store_id']],
    on=['item_id', 'store_id'],
    how='inner'
)

mapped_df = mapped_df.drop_duplicates()
mapped_intersect_df = mapped_intersect_df.drop_duplicates()

print(mapped_df.head())
print(mapped_df.shape)  # should be (176640, 2)
print(mapped_intersect_df.head())
print(mapped_intersect_df.shape)  # should be (176640, 2)

In [ ]:
his_sale5['date'] = pd.to_datetime(his_sale5['date'])
his_sale5 = his_sale5.sort_values(by='date')
print(his_sale5.head())
print(his_sale5.tail())
print(his_sale5.shape)

min_date_sale = his_sale5['date'].min() + pd.Timedelta(days=54)
max_date_sale = his_sale5['date'].max()
print(min_date_sale)
print(max_date_sale)
calendar['date'] = pd.to_datetime(calendar['date'])
calendar_trimmed = calendar[(calendar['date'] >= min_date_sale) & (calendar['date'] <= max_date_sale)].copy()
calendar_trimmed = calendar_trimmed.sort_values(by='date')
calendar_trimmed = calendar_trimmed[['date', 'week', 'holiday']].reset_index(drop=True)
print(calendar_trimmed.head())

In [ ]:
full_df1 = mapped_df.merge(calendar_trimmed, how='cross')
full_df1['sales'] = 0
full_df1['units_sold'] = 0.0

full_df2 = mapped_intersect_df.merge(calendar_trimmed, how='cross')
full_df2['sales'] = 0
full_df2['units_sold'] = 0.0

# Reorder columns to match your target format
full_df1 = full_df1[['date', 'store_id', 'item_id', 'sales', 'units_sold']]
full_df2 = full_df2[['date', 'store_id', 'item_id', 'sales', 'units_sold']]

full_df = pd.concat([full_df1, full_df2], ignore_index=True)
full_df = full_df.sort_values(by=['item_id', 'date'])

print(full_df.head())
print(full_df.shape)

In [ ]:
sale_trim = his_sale5[(his_sale5['date'] >= min_date_sale)].copy()
print(sale_trim.head)

In [ ]:
full_df = full_df.merge(sale_trim, on=['date', 'store_id', 'item_id'], how='left', suffixes=('', '_real'))

# Replace default 0s with real values where available
full_df['sales'] = full_df['sales_real'].fillna(full_df['sales'])
full_df['units_sold'] = full_df['units_sold_real'].fillna(full_df['units_sold'])

# Drop the temporary columns
full_df.drop(columns=['sales_real', 'units_sold_real'], inplace=True)

full_df = pd.merge(full_df, calendar_trimmed[['date', 'week']], on='date', how='left')

print(full_df.head())
print(full_df.shape)

In [ ]:
full_df = full_df.sort_values(by=['item_id', 'store_id', 'date']).reset_index(drop=True)
print(full_df.head(20))
print(full_df.shape)

In [ ]:
print(full_df.tail(20))


In [ ]:
weekly_df = full_df.groupby(['store_id', 'item_id', 'week'], as_index=False).agg({
    'sales': 'sum',
    'units_sold': 'sum'
})
weekly_df = weekly_df.sort_values(by=['item_id', 'store_id']).reset_index(drop=True)

print(weekly_df.nunique())
print(weekly_df.head(30))
print(weekly_df.shape)

In [ ]:
# Count number of times each (store_id, item_id) appears
submis_counts = submis.groupby(['store_id', 'item_id']).size().reset_index(name='count')

# Count how many pairs have exactly 1, 2, 3, ... entries
for n in range(1, 11):
    count_n = (submis_counts['count'] == n).sum()
    print(f"Pairs with exactly {n} entries in submis: {count_n}")


In [ ]:
weekly_df_valid = weekly_df.groupby(['store_id', 'item_id']).filter(lambda x: len(x) >= 4)

print("Pairs with at least 4 entries in weekly_df:", weekly_df_valid[['store_id', 'item_id']].drop_duplicates().shape[0])


##จัดเตรียม Test Set


In [ ]:
test_data = full_df.drop(columns=['sales', 'units_sold'])
test_data['date'] = test_data['date'] + pd.Timedelta(days=28)
test_data['week'] += 4

print(test_data.head(20))
print(test_data.shape)

In [ ]:
test_data.tail()

In [ ]:
test_data.to_csv('test_data_for_predict.csv', index=False)

##ทดสอบ

In [ ]:
test_data['sales_predict'] = 0
test_data['units_sold_predict'] = 0

test_data = test_data.groupby(['store_id', 'item_id', 'week'], as_index=False).agg({
    'sales_predict': 'sum',
    'units_sold_predict': 'sum'
})
test_data = test_data.sort_values(by=['item_id', 'store_id']).reset_index(drop=True)
test_data = test_data.rename(columns={'week': 'week_predict'})

print(test_data.nunique())
print(test_data.head(30))
print(test_data.shape)

##Label


In [ ]:
test_data = pd.read_csv('/content/submission.csv')

new_column_names = {
    'store_id': 'store_id_check',
    'item_id': 'item_id_check',
    'week': 'week_check',
    'units_sold_predict': 'predicted_sales'
}

print(test_data.head())
print(test_data.shape)

In [ ]:
test_data = test_data.rename(columns=new_column_names)

# Keep only the renamed columns
test_data = test_data[list(new_column_names.values())]

# Check the result
print(test_data.head())
print(test_data.shape)

In [ ]:
test_data = test_data.groupby(['store_id_check', 'item_id_check', 'week_check'], as_index=False).agg({
    'predicted_sales': 'sum'
})
test_data = test_data.sort_values(by=['item_id_check', 'store_id_check']).reset_index(drop=True)
test_data = test_data.rename(columns={'week_check': 'week_predict'})

print(test_data.nunique())
print(test_data.head(30))
print(test_data.shape)

In [ ]:
test_data.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(test_data['predicted_sales'], bins=500, kde=True)
plt.title("Distribution of Predicted Sales")
plt.xlabel("Predicted Sales")
plt.ylabel("Frequency")
plt.show()


In [ ]:
label_data = pd.concat([weekly_df, test_data], axis=1)
print(label_data.head())
print(label_data.shape)
print(label_data.isnull().sum)

In [ ]:
label_data.head(30)

In [ ]:
label_data.loc[label_data['predicted_sales'] < 1.5, 'predicted_sales'] = 0

In [ ]:
# Create conditions and choices
conditions = [
    (label_data['units_sold'] > 0) & (label_data['predicted_sales'] == 0),  # Delete condition
    (label_data['units_sold'] == 0) & (label_data['predicted_sales'] > 0)   # Add condition
]

choices = ['Delete', 'Add']

# Apply conditions using np.select
label_data['label'] = np.select(conditions, choices, default='Remain')

print(label_data.head())

In [ ]:
print(label_data[31:60])
print(label_data.shape)

In [ ]:
label_data['label'].value_counts().reset_index(name='count').rename(columns={'index': 'label'})


##Submission

In [ ]:
# 1. Prepare the label data: Select only necessary columns for the merge
#    Make sure to select 'week_predict' (the correct column name)
label_info = label_data[['store_id', 'item_id', 'week_predict', 'label']].copy()

# Optional: Drop duplicates in label_info if there's a possibility of multiple labels
# for the same store/item/week_predict combination. Keep the first one found.
# label_info = label_info.drop_duplicates(subset=['store_id', 'item_id', 'week_predict'], keep='first')

# 2. Perform a left merge
#    'submis' is the left DataFrame (we want to keep all its rows)
#    'label_info' is the right DataFrame
#    Use 'left_on' and 'right_on' to specify the different week column names
submis_merged = submis.merge(
    label_info,
    how='left', # Keep all rows from 'submis'
    left_on=['store_id', 'item_id', 'YEAR_WEEK_NUMBER'], # Keys in 'submis'
    right_on=['store_id', 'item_id', 'week_predict']   # Corresponding keys in 'label_info'
)

# 3. Update 'ITEM_STATUS'
#    Use combine_first: If 'label' (from the merge) is not null, use it.
#    Otherwise, keep the original 'ITEM_STATUS'.
submis_merged['ITEM_STATUS'] = submis_merged['label'].combine_first(submis_merged['ITEM_STATUS'])

# 4. Clean up: Drop the columns added during the merge that are no longer needed
#    This includes 'label' and 'week_predict' from the right DataFrame.
submis_final = submis_merged.drop(columns=['label', 'week_predict'])

# 'submis_final' now contains the updated ITEM_STATUS
print(submis_final.head(30))
print(submis_final.shape)

submis_final.to_csv('submissionQ2.csv', index=False)

In [ ]:
print(submis_final.nunique())
print(submis_final.isnull().sum())